<a href="https://colab.research.google.com/github/winnieshi/news-recommendation/blob/main/news_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Packages



In [ ]:
!apt install libomp-dev
!pip install faiss
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (267 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [ ]:
!pip install -U deepctr
!pip install deepmatch

     |████████████████████████████████| 122kB 5.1MB/s 
     |████████████████████████████████| 112kB 7.3MB/s 
  Found existing installation: deepctr 0.8.5
    Uninstalling deepctr-0.8.5:
      Successfully uninstalled deepctr-0.8.5


In [ ]:
import pandas as pd  
import numpy as np 
from collections import defaultdict  
import os, math, warnings, math, pickle, random, gc, time,logging
from tqdm import tqdm
import tqdm.notebook as tq

import lightgbm as lgb
from gensim.models import Word2Vec
import faiss
import collections
from operator import itemgetter
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss
warnings.filterwarnings('ignore')

DeepCTR version 0.8.5 detected. Your version is 0.8.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.5


#Define functions  


###memory reduce function

In [ ]:
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                    100*(start_mem-end_mem)/start_mem,
                                                                                                    (time.time()-starttime)/60))
    return df

###Get data function

In [ ]:
def get_all_click_df(data_path='./', offline=True, train_name='train_click_log.csv', test_name='testA_click_log.csv'):
    if offline:
        all_click = pd.read_csv(data_path + train_name)
    else:
        trn_click = pd.read_csv(data_path + train_name)
        tst_click = pd.read_csv(data_path + test_name)
        all_click = trn_click.append(tst_click)   
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

def get_item_info_df(data_path):
    item_info_df = pd.read_csv(data_path + 'articles.csv')
    item_info_df = item_info_df.rename(columns={'article_id': 'click_article_id'})    
    return item_info_df

def get_item_emb_dict(data_path,save_path):
    #return {item1: emb}
    item_emb_df = pd.read_csv(data_path + 'articles_emb.csv')
    item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols])
    item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)
    item_emb_dict = dict(zip(item_emb_df['article_id'], item_emb_np))
    pickle.dump(item_emb_dict, open(save_path + 'item_content_emb.pkl', 'wb'))   
    return item_emb_dict


In [ ]:
#get {user1:[(item_id, click_timestamp)]}
#{item1:[(user_id, click_timestamp)]}
def get_user_item_time(click_df):
    #return {user1:[(item_id, click_timestamp)]}
    click_df = click_df.sort_values('click_timestamp')    
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x))                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))    
    return user_item_time_dict

def get_item_user_time_dict(click_df):
    #return {item1:[(user_id, click_timestamp)]}
    def make_user_time_pair(df):
        return list(zip(df['user_id'], df['click_timestamp']))    
    click_df = click_df.sort_values('click_timestamp')
    item_user_time_df = click_df.groupby('click_article_id')['user_id', 'click_timestamp'].apply(lambda x: make_user_time_pair(x))                                                            .reset_index().rename(columns={0: 'user_time_list'})    
    item_user_time_dict = dict(zip(item_user_time_df['click_article_id'], item_user_time_df['user_time_list']))
    return item_user_time_dict

In [ ]:
def get_item_info_dict(item_info_df):
    #item_type_dict: item_id, category_id
    #item_words_dict: item_id, words_count
    #item_created_time_dict: item_id, created_at_ts
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    item_info_df['created_at_ts'] = item_info_df[['created_at_ts']].apply(max_min_scaler)    
    item_type_dict = dict(zip(item_info_df['click_article_id'], item_info_df['category_id']))
    item_words_dict = dict(zip(item_info_df['click_article_id'], item_info_df['words_count']))
    item_created_time_dict = dict(zip(item_info_df['click_article_id'], item_info_df['created_at_ts']))    
    return item_type_dict, item_words_dict, item_created_time_dict


###Get hist and last function

In [ ]:
def get_hist_and_last_click(all_click):    
    all_click = all_click.sort_values(by=['user_id', 'click_timestamp'])
    click_last_df = all_click.groupby('user_id').tail(1)
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]
    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)
    
    return click_hist_df, click_last_df

In [ ]:
def get_user_hist_item_info_dict(all_click):
    #
    user_hist_item_typs = all_click.groupby('user_id')['category_id'].agg(set).reset_index()
    user_hist_item_typs_dict = dict(zip(user_hist_item_typs['user_id'], user_hist_item_typs['category_id']))
    user_hist_item_ids_dict = all_click.groupby('user_id')['click_article_id'].agg(set).reset_index()
    user_hist_item_ids_dict = dict(zip(user_hist_item_ids_dict['user_id'], user_hist_item_ids_dict['click_article_id']))
    user_hist_item_words = all_click.groupby('user_id')['words_count'].agg('mean').reset_index()
    user_hist_item_words_dict = dict(zip(user_hist_item_words['user_id'], user_hist_item_words['words_count']))
    all_click_ = all_click.sort_values('click_timestamp')
    user_last_item_created_time = all_click_.groupby('user_id')['created_at_ts'].apply(lambda x: x.iloc[-1]).reset_index()
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    user_last_item_created_time['created_at_ts'] = user_last_item_created_time[['created_at_ts']].apply(max_min_scaler)
    user_last_item_created_time_dict = dict(zip(user_last_item_created_time['user_id'],user_last_item_created_time['created_at_ts']))
    return user_hist_item_typs_dict, user_hist_item_ids_dict, user_hist_item_words_dict, user_last_item_created_time_dict


###Get top popular item function

In [ ]:
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

In [ ]:
#Get top popular items for a specific day. 
def day_topItem_to_dict(all_click):
  topk_click = all_click.groupby(['click_year','click_month','click_day','click_article_id'])['user_id'].count().reset_index()
  topk_click['popular_day_rank'] = topk_click.groupby(['click_year','click_month','click_day'])['user_id'].rank(ascending=False,method = 'first')
  topk_click = topk_click[topk_click['popular_day_rank']<=50]
  def f(x):
    return str(x['click_year'])+str(x['click_month'])+str(x['click_day'])
  topk_click['specific_day'] = topk_click.apply(f,axis=1)
  day_topItem_df = topk_click.groupby('specific_day')['click_article_id'].apply(lambda x:[i for i in x]).reset_index()
  day_topItem_dict = dict(zip(day_topItem_df['specific_day'],day_topItem_df['click_article_id']))
  pickle.dump(day_topItem_dict, open(save_path + 'day_topItem_dict.pkl', 'wb'))   
  return day_topItem_dict
def get_item_topk_click_day(click_df,all_click_df,user, k=10):
  if os.path.exists(save_path+'day_topItem_dict.pkl'):
    day_topItem_dict=pickle.load(open(save_path + 'day_topItem_dict.pkl', 'rb'))
  else:
    day_topItem_dict=day_topItem_to_dict(all_click_df)
  user_last= click_df[click_df['user_id']==user].sort_values('click_timestamp').tail(1)
  user_last_day = str(user_last['click_year'].values[0])+str(user_last['click_month'].values[0])+str(user_last['click_day'].values[0])
  topk_item_days = day_topItem_dict[user_last_day][:k]
  return topk_item_days

##Metric method

In [ ]:
def metrics_recall(user_recall_items_dict, trn_last_click_df, topk=50):
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['click_article_id']))
    user_num = len(user_recall_items_dict) 
    for k in range(5, topk+1, 5):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            tmp_recall_items = [x[0] for x in user_recall_items_dict[user][:k]]
            if last_click_item_dict[user] in set(tmp_recall_items):
                hit_num += 1        
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)

##Itemcf_similarity

In [ ]:
def itemcf_sim(save_path, df, item_created_time_dict, offline= True):
    user_item_time_dict = get_user_item_time(df)
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tq.tqdm(user_item_time_dict.items()):
        for loc1, (i, i_click_time) in enumerate(item_time_list):
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for loc2, (j, j_click_time) in enumerate(item_time_list):
                if(i == j):
                    continue
                loc_alpha = 1.0 if loc2 > loc1 else 0.7
                # loc_weight = loc_alpha * (0.7 ** (np.abs(loc2 - loc1) - 1))
                click_time_weight = np.exp(0.5 ** np.abs(i_click_time - j_click_time))
                created_time_weight = np.exp(0.9 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
                i2i_sim[i].setdefault(j, 0)
                i2i_sim[i][j] += click_time_weight * created_time_weight / math.log(len(item_time_list) + 1)                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    if offline:
        pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim_7_5_6_notest.pkl', 'wb'))
    else:
        pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim_7_5_6_havetest.pkl', 'wb')) 
    return i2i_sim_

In [ ]:
i2i_sim = itemcf_sim(save_path, all_click_df, item_created_time_dict, offline=False)

NameError: ignored

##Item embedding simiarity

In [ ]:
def embdding_sim(click_df, item_emb_df, save_path, topk):
    item_idx_2_rawid_dict = dict(zip(item_emb_df.index, item_emb_df['article_id']))    
    item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols].values, dtype=np.float32)
    item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)
    item_index = faiss.IndexFlatIP(item_emb_np.shape[1])
    item_index.add(item_emb_np)
    sim, idx = item_index.search(item_emb_np, topk) 
    item_sim_dict = collections.defaultdict(dict)
    for target_idx, sim_value_list, rele_idx_list in tq.tqdm(zip(range(len(item_emb_np)), sim, idx)):
        target_raw_id = item_idx_2_rawid_dict[target_idx]
        for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1:]): 
            rele_raw_id = item_idx_2_rawid_dict[rele_idx]
            item_sim_dict[target_raw_id][rele_raw_id] = item_sim_dict.get(target_raw_id, {}).get(rele_raw_id, 0) + sim_value
    pickle.dump(item_sim_dict, open(save_path + 'emb_i2i_sim.pkl', 'wb'))       
    return item_sim_dict

##Item based recommend
get item based recommendation according to content similarity, created time, click time, topk item. 

In [ ]:
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click, item_created_time_dict, emb_i2i_sim):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
        :param i2i_sim: 字典，文章相似性矩阵
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全
        :param emb_i2i_sim: 字典基于内容embedding算的文章相似矩阵
        
        return: 召回的文章列表 [(item1, score1), (item2, score2)...]
    """
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    user_hist_items_ = {user_id for user_id, _ in user_hist_items}
    
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items_:
                continue
            
            # 文章创建时间差权重
            created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
            # 相似文章和历史点击文章序列中历史文章所在的位置权重
            loc_weight = (0.9 ** (len(user_hist_items) - loc))
            
            content_weight = 1.0
            if emb_i2i_sim.get(i, {}).get(j, None) is not None:
                content_weight += emb_i2i_sim[i][j]
            if emb_i2i_sim.get(j, {}).get(i, None) is not None:
                content_weight += emb_i2i_sim[j][i]
                
            item_rank.setdefault(j, 0)
            item_rank[j] += created_time_weight * loc_weight * content_weight * wij
    
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank

In [ ]:
def item_based_recommend2(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click, item_created_time_dict, emb_i2i_sim):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
        :param i2i_sim: 字典，文章相似性矩阵
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全
        :param emb_i2i_sim: 字典基于内容embedding算的文章相似矩阵
        
        return: 召回的文章列表 [(item1, score1), (item2, score2)...]
    """
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    user_hist_items_ = {item_id for item_id, _ in user_hist_items}
    
    item_rank = {}
    #先把top20放进去
    for i, item in enumerate(item_topk_click[:20]):
        if item in item_rank.items(): # 填充的item应该不在原来的列表中
            continue
        item_rank[item] = 100 # 随便给个负数就行
    # 再补充相似item
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items_:
                continue
            
            # 文章创建时间差权重
            created_time_weight = np.exp(0.9**np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
            # 相似文章和历史点击文章序列中历史文章所在的位置权重
            # loc_weight = (0.9 ** (len(user_hist_items) - loc))
            
            content_weight = 1.0
            if emb_i2i_sim.get(i, {}).get(j, None) is not None:
                content_weight += emb_i2i_sim[i][j]
            if emb_i2i_sim.get(j, {}).get(i, None) is not None:
                content_weight += emb_i2i_sim[j][i]
                
            item_rank.setdefault(j, 0)
            item_rank[j] += created_time_weight * content_weight * wij
    
    # 不足50个，用热门商品补全
    if len(item_rank) < recall_item_num:
      for i, item in enumerate(item_topk_click[20:]):
          if item in item_rank.items(): # 填充的item应该不在原来的列表中
              continue
          item_rank[item] = 100 # 随便给个负数就行
          if len(item_rank) == recall_item_num:
              break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank

##Get_recall function

In [ ]:
def get_recall(save_path,all_click_df,item_created_time_dict, metric_recall=True):
  #return dict: {user1:[(item1, score1), (item2, score2)...]...}
    if metric_recall:
        trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
    else:
        trn_hist_click_df = all_click_df
    user_recall_items_dict = collections.defaultdict(dict)
    user_item_time_dict = get_user_item_time(trn_hist_click_df)
    if os.path.exists(save_path + 'itemcf_i2i_sim_7_5_6_havetest.pkl'):
        i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim_7_5_6_havetest.pkl', 'rb'))
    else:
        i2i_sim = itemcf_sim(save_path, all_click_df, item_created_time_dict, offline=False)

    item_emb_df = pd.read_csv(data_path + 'articles_emb.csv')
    if os.path.exists(save_path + 'emb_i2i_sim.pkl'):
        emb_i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl', 'rb'))
    else:
        emb_i2i_sim = embdding_sim(all_click_df, item_emb_df, save_path, topk=10) 

    sim_item_topk = 50
    recall_item_num = 50
    item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

    for user in tq.tqdm(trn_hist_click_df['user_id'].unique()):
        user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict,i2i_sim, sim_item_topk, \
                            recall_item_num,item_topk_click, item_created_time_dict, emb_i2i_sim)
    if metric_recall:
        metrics_recall(user_recall_items_dict, trn_last_click_df, topk=recall_item_num)
        pickle.dump(user_recall_items_dict, open(save_path + 'itemcf_recall_dict_756_001_3_250_topk50_num10_notest.pkl', 'wb'))
    else:
        pickle.dump(user_recall_items_dict, open(save_path + 'itemcf_recall_dict_756_001_3_250_topk50_num10_havetest.pkl', 'wb'))

In [ ]:

def get_recall_day(save_path,click_df,all_click_df,item_created_time_dict, metric_recall=True):
  #return dict: {user1:[(item1, score1), (item2, score2)...]...}
    if metric_recall:
        trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(click_df)
    else:
        trn_hist_click_df = click_df
    user_recall_items_dict = collections.defaultdict(dict)
    user_item_time_dict = get_user_item_time(trn_hist_click_df)

    if os.path.exists(save_path + 'itemcf_i2i_sim_7_5_6_havetest.pkl'):
        i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim_7_5_6_havetest.pkl', 'rb'))
    else:
        i2i_sim = itemcf_sim(save_path, all_click_df, item_created_time_dict, offline=False)

    item_emb_df = pd.read_csv(data_path + 'articles_emb.csv')
    if os.path.exists(save_path + 'emb_i2i_sim.pkl'):
        emb_i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl', 'rb'))
    else:
        emb_i2i_sim = embdding_sim(all_click_df, item_emb_df, save_path, topk=10) 

    sim_item_topk = 50
    recall_item_num = 50
    item_topk_click = get_item_topk_click(all_click_df, k=50)

    for user in tq.tqdm(trn_hist_click_df['user_id'].unique()):
        item_topk_click_day = get_item_topk_click_day(trn_hist_click_df, all_click_df ,user, k=20)
        item_topk_click_day.extend(item_topk_click)
        user_recall_items_dict[user] = item_based_recommend2(user, user_item_time_dict,i2i_sim, sim_item_topk, \
                            recall_item_num,item_topk_click_day, item_created_time_dict, emb_i2i_sim)
    if metric_recall:
        metrics_recall(user_recall_items_dict, trn_last_click_df, topk=recall_item_num)
        pickle.dump(user_recall_items_dict, open(save_path + 'itemcf_recall_dict_recall50_hotItemDay_no_last.pkl', 'wb'))
    else:
        pickle.dump(user_recall_items_dict, open(save_path + 'itemcf_recall_dict_recall50_hotItemDay.pkl', 'wb'))

In [ ]:
# get_recall_day('./temp/',all_click_df_trn_val, all_click_df, item_created_time_dict, metric_recall=True)
# # get_recall_day(save_path, all_click_df_trn_val, all_click_df, item_created_time_dict, metric_recall=True)
#  topk:  5  :  hit_num:  34332 hit_rate:  0.17166 user_num :  200000
#  topk:  10  :  hit_num:  37975 hit_rate:  0.18987 user_num :  200000
#  topk:  15  :  hit_num:  38940 hit_rate:  0.1947 user_num :  200000
#  topk:  20  :  hit_num:  43191 hit_rate:  0.21596 user_num :  200000
#  topk:  25  :  hit_num:  95482 hit_rate:  0.47741 user_num :  200000
#  topk:  30  :  hit_num:  114310 hit_rate:  0.57155 user_num :  200000
#  topk:  35  :  hit_num:  126026 hit_rate:  0.63013 user_num :  200000
#  topk:  40  :  hit_num:  134124 hit_rate:  0.67062 user_num :  200000
#  topk:  45  :  hit_num:  140290 hit_rate:  0.70145 user_num :  200000
#  topk:  50  :  hit_num:  148020 hit_rate:  0.7401 user_num :  200000

In [ ]:
# click_val

In [ ]:
click_trn_val_users = click_trn['user_id'].unique().tolist()
all_click_df_trn_val = all_click_df[all_click_df['user_id'].isin(click_trn_val_users)]

In [ ]:
sample_user_ids = np.random.choice(all_click_df['user_id'].unique(), size=5000, replace=False) 
sample_trn = all_click_df[all_click_df['user_id'].isin(sample_user_ids)]

In [ ]:
get_recall_day(save_path,sample_trn, all_click_df, item_created_time_dict, metric_recall=True)



 topk:  5  :  hit_num:  867 hit_rate:  0.1734 user_num :  5000
 topk:  10  :  hit_num:  1037 hit_rate:  0.2074 user_num :  5000
 topk:  15  :  hit_num:  1123 hit_rate:  0.2246 user_num :  5000
 topk:  20  :  hit_num:  1250 hit_rate:  0.25 user_num :  5000
 topk:  25  :  hit_num:  2178 hit_rate:  0.4356 user_num :  5000
 topk:  30  :  hit_num:  2501 hit_rate:  0.5002 user_num :  5000
 topk:  35  :  hit_num:  2687 hit_rate:  0.5374 user_num :  5000
 topk:  40  :  hit_num:  2837 hit_rate:  0.5674 user_num :  5000
 topk:  45  :  hit_num:  2954 hit_rate:  0.5908 user_num :  5000
 topk:  50  :  hit_num:  3109 hit_rate:  0.6218 user_num :  5000


In [ ]:
# #hot_day
# metrics_recall(sample_user_recall_items_dict, sample_last, topk=80)
#  topk:  5  :  hit_num:  2991 hit_rate:  0.2991 user_num :  10000
#  topk:  10  :  hit_num:  4106 hit_rate:  0.4106 user_num :  10000
#  topk:  15  :  hit_num:  4749 hit_rate:  0.4749 user_num :  10000
#  topk:  20  :  hit_num:  5245 hit_rate:  0.5245 user_num :  10000
#  topk:  25  :  hit_num:  5610 hit_rate:  0.561 user_num :  10000
#  topk:  30  :  hit_num:  5914 hit_rate:  0.5914 user_num :  10000
#  topk:  35  :  hit_num:  6155 hit_rate:  0.6155 user_num :  10000
#  topk:  40  :  hit_num:  6359 hit_rate:  0.6359 user_num :  10000
#  topk:  45  :  hit_num:  6546 hit_rate:  0.6546 user_num :  10000
#  topk:  50  :  hit_num:  6711 hit_rate:  0.6711 user_num :  10000
#  topk:  55  :  hit_num:  6849 hit_rate:  0.6849 user_num :  10000
#  topk:  60  :  hit_num:  6978 hit_rate:  0.6978 user_num :  10000
#  topk:  65  :  hit_num:  7084 hit_rate:  0.7084 user_num :  10000
#  topk:  70  :  hit_num:  7188 hit_rate:  0.7188 user_num :  10000
#  topk:  75  :  hit_num:  7269 hit_rate:  0.7269 user_num :  10000
#  topk:  80  :  hit_num:  7330 hit_rate:  0.733 user_num :  10000

In [ ]:
# metrics_recall(user_recall_items_dict, trn_last_click_df, topk=50)
#  topk:  5  :  hit_num:  70892 hit_rate:  0.35446 user_num :  200000
#  topk:  10  :  hit_num:  96331 hit_rate:  0.48165 user_num :  200000
#  topk:  15  :  hit_num:  96331 hit_rate:  0.48165 user_num :  200000
#  topk:  20  :  hit_num:  96331 hit_rate:  0.48165 user_num :  200000
#  topk:  25  :  hit_num:  96331 hit_rate:  0.48165 user_num :  200000
#  topk:  30  :  hit_num:  96331 hit_rate:  0.48165 user_num :  200000
#  topk:  35  :  hit_num:  96331 hit_rate:  0.48165 user_num :  200000
#  topk:  40  :  hit_num:  96331 hit_rate:  0.48165 user_num :  200000
#  topk:  45  :  hit_num:  96331 hit_rate:  0.48165 user_num :  200000
#  topk:  50  :  hit_num:  96331 hit_rate:  0.48165 user_num :  200000

In [ ]:
# metrics_recall(user_recall_items_dict, trn_last_click_df, topk=50)
#  topk:  5  :  hit_num:  70797 hit_rate:  0.35398 user_num :  200000
#  topk:  10  :  hit_num:  96222 hit_rate:  0.48111 user_num :  200000
#  topk:  15  :  hit_num:  111675 hit_rate:  0.55837 user_num :  200000
#  topk:  20  :  hit_num:  122392 hit_rate:  0.61196 user_num :  200000
#  topk:  25  :  hit_num:  130169 hit_rate:  0.65085 user_num :  200000
#  topk:  30  :  hit_num:  136404 hit_rate:  0.68202 user_num :  200000
#  topk:  35  :  hit_num:  141520 hit_rate:  0.7076 user_num :  200000
#  topk:  40  :  hit_num:  145990 hit_rate:  0.72995 user_num :  200000
#  topk:  45  :  hit_num:  149706 hit_rate:  0.74853 user_num :  200000
#  topk:  50  :  hit_num:  152836 hit_rate:  0.76418 user_num :  200000

In [ ]:
# sample_user_recall_items_dict = collections.defaultdict(dict)
# sample_user_item_time_dict = get_user_item_time(sample_hist)
# # i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim_7_5_6_havetest.pkl', 'rb'))
# # emb_i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl', 'rb'))
# sim_item_topk = 50
# recall_item_num = 50
# item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

# for user in tq.tqdm(sample_hist['user_id'].unique()):
#     # item_topk_click_day = get_item_topk_click_day(sample_hist, all_click_df ,user, k=10)
#     # item_topk_click_day.extend(item_topk_click)
#     sample_user_recall_items_dict[user] = item_based_recommend(user, sample_user_item_time_dict,i2i_sim, sim_item_topk, \
#                         recall_item_num,item_topk_click, item_created_time_dict, emb_i2i_sim)


In [ ]:
# #original
# metrics_recall(sample_user_recall_items_dict, sample_last, topk=50)
# :  5  :  hit_num:  2973 hit_rate:  0.2973 user_num :  10000
#  topk:  10  :  hit_num:  4090 hit_rate:  0.409 user_num :  10000
#  topk:  15  :  hit_num:  4728 hit_rate:  0.4728 user_num :  10000
#  topk:  20  :  hit_num:  5207 hit_rate:  0.5207 user_num :  10000
#  topk:  25  :  hit_num:  5587 hit_rate:  0.5587 user_num :  10000
#  topk:  30  :  hit_num:  5881 hit_rate:  0.5881 user_num :  10000
#  topk:  35  :  hit_num:  6109 hit_rate:  0.6109 user_num :  10000
#  topk:  40  :  hit_num:  6340 hit_rate:  0.634 user_num :  10000
#  topk:  45  :  hit_num:  6522 hit_rate:  0.6522 user_num :  10000
#  topk:  50  :  hit_num:  6670 hit_rate:  0.667 user_num :  10000

 topk:  5  :  hit_num:  2972 hit_rate:  0.2972 user_num :  10000
 topk:  10  :  hit_num:  4088 hit_rate:  0.4088 user_num :  10000
 topk:  15  :  hit_num:  4727 hit_rate:  0.4727 user_num :  10000
 topk:  20  :  hit_num:  5205 hit_rate:  0.5205 user_num :  10000
 topk:  25  :  hit_num:  5585 hit_rate:  0.5585 user_num :  10000
 topk:  30  :  hit_num:  5880 hit_rate:  0.588 user_num :  10000
 topk:  35  :  hit_num:  6108 hit_rate:  0.6108 user_num :  10000
 topk:  40  :  hit_num:  6339 hit_rate:  0.6339 user_num :  10000
 topk:  45  :  hit_num:  6521 hit_rate:  0.6521 user_num :  10000
 topk:  50  :  hit_num:  6670 hit_rate:  0.667 user_num :  10000

#User features

In [ ]:
def create_feature(users_id, recall_list, click_hist_df,  articles_info, articles_emb, user_emb=None, N=1):
    #return user_id,click_article_id, sim_0,..sim_i, time_diff_i, word_diff_i, sim_max,sim_min,sim_sum,sim_mean,score,rank,label
    all_user_feas = []
    i = 0
    for user_id in tq.tqdm(users_id):
        hist_user_items = click_hist_df[click_hist_df['user_id']==user_id]['click_article_id'][-N:]
        for rank, (article_id, score, label) in enumerate(recall_list[user_id]):
            a_create_time = articles_info[articles_info['article_id']==article_id]['created_at_ts'].values[0]
            a_words_count = articles_info[articles_info['article_id']==article_id]['words_count'].values[0]
            single_user_fea = [user_id, article_id]
            sim_fea = []
            time_fea = []
            word_fea = []
            for hist_item in hist_user_items:
                b_create_time = articles_info[articles_info['article_id']==hist_item]['created_at_ts'].values[0]
                b_words_count = articles_info[articles_info['article_id']==hist_item]['words_count'].values[0]
                sim_fea.append(np.dot(articles_emb[hist_item], articles_emb[article_id]))
                time_fea.append(abs(a_create_time-b_create_time))
                word_fea.append(abs(a_words_count-b_words_count))
            single_user_fea.extend(sim_fea)  
            single_user_fea.extend(time_fea)   
            single_user_fea.extend(word_fea)  
            single_user_fea.extend([max(sim_fea), min(sim_fea), sum(sim_fea), sum(sim_fea) / len(sim_fea)])  
            if user_emb:  
                single_user_fea.append(np.dot(user_emb[user_id], articles_emb[article_id]))       
            single_user_fea.extend([score, rank, label])    
            all_user_feas.append(single_user_fea)
    id_cols = ['user_id', 'click_article_id']
    sim_cols = ['sim' + str(i) for i in range(N)]
    time_cols = ['time_diff' + str(i) for i in range(N)]
    word_cols = ['word_diff' + str(i) for i in range(N)]
    sat_cols = ['sim_max', 'sim_min', 'sim_sum', 'sim_mean']
    user_item_sim_cols = ['user_item_sim'] if user_emb else []
    user_score_rank_label = ['score', 'rank', 'label']
    cols = id_cols + sim_cols + time_cols + word_cols + sat_cols + user_item_sim_cols + user_score_rank_label
    df = pd.DataFrame( all_user_feas, columns=cols)
    return df

##User activity
according to the number of clicks and time difference between consecutive clicks. 

In [ ]:
def active_level(all_data, cols):
    #return df, cols;=['user_id','click_size','active_level']
    data = all_data[cols]
    data.sort_values(['user_id', 'click_timestamp'], inplace=True)
    user_act = pd.DataFrame(data.groupby('user_id', as_index=False)[['click_article_id', 'click_timestamp']].\
            agg({'click_article_id':np.size, 'click_timestamp': {list}}).values, columns=['user_id', 'click_size', 'click_timestamp'])
    def time_diff_mean(l):
        if len(l) == 1:
            return 1
        else:
            return np.mean([j-i for i, j in list(zip(l[:-1], l[1:]))])
    user_act['time_diff_mean'] = user_act['click_timestamp'].apply(lambda x: time_diff_mean(x))
    # user_act['click_size'] = 1 / user_act['click_size']
    user_act['click_size'] = (user_act['click_size'] - user_act['click_size'].min()) / (user_act['click_size'].max() - user_act['click_size'].min())
    user_act['time_diff_mean'] = (user_act['time_diff_mean'] - user_act['time_diff_mean'].min()) / (user_act['time_diff_mean'].max() - user_act['time_diff_mean'].min())     
    user_act['active_level'] = user_act['click_size'] + user_act['time_diff_mean']
    user_act['user_id'] = user_act['user_id'].astype('int')
    del user_act['click_timestamp']
    return user_act

##User device feature

In [ ]:
def device_fea(all_data, cols):
    user_device_info = all_data[cols]
    user_device_info = user_device_info.groupby('user_id').agg(lambda x: x.value_counts().index[0]).reset_index()
    return user_device_info

##User time preferrence feature

In [ ]:
def user_time_hob_fea(all_data, cols):
  #return average of click_timestamp and average of clicked articles'created timestamp
    user_time_hob_info = all_data[cols]
    mm = MinMaxScaler()
    user_time_hob_info['click_timestamp'] = mm.fit_transform(user_time_hob_info[['click_timestamp']])
    user_time_hob_info['created_at_ts'] = mm.fit_transform(user_time_hob_info[['created_at_ts']])
    user_time_hob_info = user_time_hob_info.groupby('user_id').agg('mean').reset_index()
    user_time_hob_info.rename(columns={'click_timestamp': 'user_time_hob1', 'created_at_ts': 'user_time_hob2'}, inplace=True)
    return user_time_hob_info

##User category preferrence feature:


In [ ]:
def user_cat_hob_fea(all_data, cols):
    #reutrn user_id,categroy list
    user_category_hob_info = all_data[cols]
    user_category_hob_info = user_category_hob_info.groupby('user_id').agg({list}).reset_index() 
    user_cat_hob_info = pd.DataFrame()
    user_cat_hob_info['user_id'] = user_category_hob_info['user_id']
    user_cat_hob_info['cate_list'] = user_category_hob_info['category_id']    
    return user_cat_hob_info

##Article hot level


In [ ]:
def hot_level(all_data, cols):
    #return ariticle_hot df: cols=['click_article_id', 'user_num','hot_level']
    data = all_data[cols]
    data.sort_values(['click_article_id', 'click_timestamp'], inplace=True)
    article_hot = pd.DataFrame(data.groupby('click_article_id', as_index=False)[['user_id', 'click_timestamp']].\
                          agg({'user_id':np.size, 'click_timestamp': {list}}).values, columns=['click_article_id', 'user_num', 'click_timestamp'])
    def time_diff_mean(l):
        if len(l) == 1:
            return 1
        else:
            return np.mean([j-i for i, j in list(zip(l[:-1], l[1:]))])
    article_hot['time_diff_mean'] = article_hot['click_timestamp'].apply(lambda x: time_diff_mean(x))
    # article_hot['user_num'] = 1 / article_hot['user_num']
    article_hot['user_num'] = (article_hot['user_num'] - article_hot['user_num'].min()) / (article_hot['user_num'].max() - article_hot['user_num'].min())
    article_hot['time_diff_mean'] = (article_hot['time_diff_mean'] - article_hot['time_diff_mean'].min()) / (article_hot['time_diff_mean'].max() - article_hot['time_diff_mean'].min())     
    article_hot['hot_level'] = article_hot['user_num'] + article_hot['time_diff_mean']
    article_hot['click_article_id'] = article_hot['click_article_id'].astype('int')
    del article_hot['click_timestamp']
    return article_hot

#Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Data Science/news_rec
!ls



Mounted at /content/drive
/content/drive/MyDrive/Data Science/news_rec
'DATA ETL.ipynb'				      news_rec_youtubeDNN.ipynb
 itemcf_recall_dict_recall50_hotItemDay_no_last.pkl   prediction_result
 itemcf_recall_dict_recall50_hotItemDay.pkl	      tcdata
 news_rec_multiple_recall.ipynb			      user_data


In [ ]:
import pandas as pd  
import numpy as np 
import os, pickle
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

In [ ]:
data_path = './tcdata/'
save_path = './user_data/'
pred_path ='./prediction_result/'

In [ ]:
train_name='train_click_log.csv'
test_name='testA_click_log.csv'
article_name='articles.csv'
user_info_name='user_info.csv'

In [ ]:
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
import time
import datetime
all_click_df = get_all_click_df(data_path, offline=False,train_name=train_name, test_name=test_name)
all_click_df['click_year'] = all_click_df['click_timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%Y'))
all_click_df['click_month'] = all_click_df['click_timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%m'))
all_click_df['click_day'] = all_click_df['click_timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%d'))
all_click_df['click_timestamp'] = all_click_df[['click_timestamp']].apply(max_min_scaler)

In [ ]:
item_info_df = get_item_info_df(data_path)

if os.path.exists(save_path + 'item_content_emb.pkl'):
    item_emb_dict=pickle.load(open(save_path + 'item_content_emb.pkl', 'rb'))
else:
    item_emb_dict = get_item_emb_dict(data_path,save_path)

In [ ]:
item_type_dict, item_words_dict, item_created_time_dict = get_item_info_dict(item_info_df)

#Rank recall result-LGB RANK MODEL

##Prepare dataset train val test

In [ ]:
def trn_val_split(all_click_df, sample_user_nums=10000):
    all_click = all_click_df
    all_user_ids = all_click.user_id.unique()
    sample_user_ids = np.random.choice(all_user_ids, size=sample_user_nums, replace=False) 
    click_val = all_click[all_click['user_id'].isin(sample_user_ids)]
    click_trn = all_click[~all_click['user_id'].isin(sample_user_ids)]
    click_val = click_val.sort_values(['user_id', 'click_timestamp'])
    val_ans = click_val.groupby('user_id').tail(1)
    click_val = click_val.groupby('user_id').apply(lambda x: x[:-1]).reset_index(drop=True)
    val_ans = val_ans[val_ans.user_id.isin(click_val.user_id.unique())]
    click_val = click_val[click_val.user_id.isin(val_ans.user_id.unique())]
    return click_trn, click_val, val_ans

In [ ]:
have_validation = True
def get_trn_val_tst_data(data_path, train_name, test_name,have_validation=True):
    if have_validation:
        click_trn_data = pd.read_csv(data_path + train_name)  
        click_trn_data = reduce_mem(click_trn_data)
        click_trn, click_val, val_ans = trn_val_split(click_trn_data, sample_user_nums=10000)
    else:
        click_trn = pd.read_csv(data_path + train_name)
        click_trn = reduce_mem(click_trn)
        click_val = None
        val_ans = None
    click_tst = pd.read_csv(data_path + test_name)   
    return click_trn, click_val, click_tst, val_ans

###Split data to trn, val,tst dataset

In [ ]:
click_trn, click_val, click_tst, val_ans = get_trn_val_tst_data(data_path,train_name,test_name,have_validation)

-- Mem. usage decreased to 23.34 Mb (69.4% reduction),time spend:0.00 min


In [ ]:
val_ans['user_id'].unique()[:100]

array([   5,    9,   47,   58,   75,   82,  101,  113,  137,  138,  139,
        154,  164,  165,  190,  207,  263,  311,  335,  384,  388,  399,
        407,  413,  496,  527,  532,  534,  538,  559,  628,  711,  712,
        721,  799,  813,  828,  884,  919,  927,  976, 1009, 1018, 1044,
       1049, 1079, 1107, 1137, 1145, 1164, 1171, 1181, 1199, 1213, 1235,
       1245, 1258, 1298, 1307, 1312, 1344, 1347, 1392, 1433, 1508, 1509,
       1521, 1525, 1535, 1554, 1560, 1606, 1627, 1635, 1648, 1655, 1663,
       1678, 1737, 1750, 1759, 1762, 1776, 1814, 1817, 1831, 1850, 1904,
       1918, 1939, 1941, 1981, 1988, 2027, 2075, 2088, 2117, 2133, 2152,
       2158], dtype=int32)

###Get recall from itemcf_similarity

In [ ]:
#For trn and val users, get_recall for training. 
click_trn_val_users = click_trn['user_id'].unique().tolist()+click_val['user_id'].unique().tolist()
all_click_df_trn_val = all_click_df[all_click_df['user_id'].isin(click_trn_val_users)]
if os.path.exists(save_path + 'itemcf_recall_dict_recall50_hotItemDay_no_last.pkl'):
    pass
else:
    get_recall_day(save_path, all_click_df_trn_val, all_click_df, item_created_time_dict, metric_recall=True)
#For tst users, get_recall for test, so include last click. 
click_tst_users = click_tst['user_id'].unique()
all_click_df_tst = all_click_df[all_click_df['user_id'].isin(click_tst_users)]
if os.path.exists(save_path + 'itemcf_recall_dict_recall50_hotItemDay.pkl'):
    pass
else:
    get_recall_day(save_path, all_click_df_tst, all_click_df, item_created_time_dict, metric_recall=False)


 topk:  5  :  hit_num:  34332 hit_rate:  0.17166 user_num :  200000
 topk:  10  :  hit_num:  37975 hit_rate:  0.18987 user_num :  200000
 topk:  15  :  hit_num:  38940 hit_rate:  0.1947 user_num :  200000
 topk:  20  :  hit_num:  43191 hit_rate:  0.21596 user_num :  200000
 topk:  25  :  hit_num:  95482 hit_rate:  0.47741 user_num :  200000
 topk:  30  :  hit_num:  114310 hit_rate:  0.57155 user_num :  200000
 topk:  35  :  hit_num:  126026 hit_rate:  0.63013 user_num :  200000
 topk:  40  :  hit_num:  134124 hit_rate:  0.67062 user_num :  200000
 topk:  45  :  hit_num:  140290 hit_rate:  0.70145 user_num :  200000
 topk:  50  :  hit_num:  148020 hit_rate:  0.7401 user_num :  200000


In [ ]:
if os.path.exists("./user_data/datastore/all_click_df_hist.csv") and os.path.exists("./user_data/datastore/all_click_df_last.csv"):
    all_click_df_hist=pd.read_csv("./user_data/datastore/all_click_df_hist.csv")
    all_click_df_last=pd.read_csv("./user_data/datastore/all_click_df_last.csv")
else:
    all_click_df_hist, all_click_df_last = get_hist_and_last_click(all_click_df)
    all_click_df_hist.to_csv("./user_data/datastore/all_click_df_hist.csv",index=False)
    all_click_df_last.to_csv("./user_data/datastore/all_click_df_last.csv",index=False)
    
click_trn_hist = all_click_df_hist[all_click_df_hist['user_id'].isin(click_trn['user_id'].unique())]
click_trn_last = all_click_df_last[all_click_df_last['user_id'].isin(click_trn['user_id'].unique())]
if click_val is not None:
    click_val_hist, click_val_last = click_val, val_ans
else:
    click_val_hist, click_val_last = None, None    
click_tst_hist = click_tst

###Get recall_list from document

In [ ]:
def recall_dict_2_df(recall_list_dict):
    df_row_list = [] 
    for user, recall_list in tq.tqdm(recall_list_dict.items()):
        for item, score in recall_list:
            df_row_list.append([user, item, score])
    col_names = ['user_id', 'sim_item', 'score']
    recall_list_df = pd.DataFrame(df_row_list, columns=col_names)    
    return recall_list_df

recall_list_dict_50_nolast = pickle.load(open(save_path + 'itemcf_recall_dict_recall50_hotItemDay_no_last.pkl', 'rb'))
recall_list_df50_nolast = recall_dict_2_df(recall_list_dict_50_nolast)
recall_list_dict_50 = pickle.load(open(save_path + 'itemcf_recall_dict_recall50_hotItemDay.pkl', 'rb'))
recall_list_df50 = recall_dict_2_df(recall_list_dict_50)


###Label the candidates and get neg samples

In [ ]:
def get_rank_label_df(recall_list_df, label_df, is_test=False):
  #recall_list_df input: ['user_id', 'sim_item', 'score']
  #output: ['user_id', 'sim_item', 'score','label'] 
  #label=1 if hit!, label=0 is not hit, label=-1 is test dataset

    if is_test:
        recall_list_df['label'] = -1
        return recall_list_df
    label_df = label_df.rename(columns={'click_article_id': 'sim_item'})
    recall_list_df_ = recall_list_df.merge(label_df[['user_id', 'sim_item', 'click_timestamp']],how='left', on=['user_id', 'sim_item'])
    recall_list_df_['label'] = recall_list_df_['click_timestamp'].apply(lambda x: 0.0 if np.isnan(x) else 1.0)
    del recall_list_df_['click_timestamp']
    return recall_list_df_
    
def neg_sample_recall_data(recall_items_df, sample_rate=0.001):
  #keep all pos sample 
  #for each user_id, get at least 1 and at most 5 neg sample
  #for each item, get at least 1 and at most 5 neg sample
    pos_data = recall_items_df[recall_items_df['label'] == 1]
    neg_data = recall_items_df[recall_items_df['label'] == 0]   
    print('pos_data_num:', len(pos_data), 'neg_data_num:', len(neg_data), 'pos/neg:', len(pos_data)/len(neg_data))    
    def neg_sample_func(group_df):
        sample_num =5
        neg_num = len(group_df)
        sample_num = max(int(neg_num * sample_rate), 1)
        sample_num = min(sample_num, 5) 
        return group_df.sample(n=sample_num, replace=True)
    neg_data_user_sample = neg_data.groupby('user_id', group_keys=False).apply(neg_sample_func)
    neg_data_item_sample = neg_data.groupby('sim_item', group_keys=False).apply(neg_sample_func)
    neg_data_new = neg_data_user_sample.append(neg_data_item_sample)
    neg_data_new = neg_data_new.sort_values(['user_id', 'score']).drop_duplicates(['user_id', 'sim_item'], keep='last')
    data_new = pd.concat([pos_data, neg_data_new], ignore_index=True)
    return data_new

def get_user_recall_item_label_df(click_trn_hist, click_val_hist, click_tst_hist,click_trn_last, click_val_last, recall_list_df_nolast, recall_list_df,click_val):
    trn_user_items_df = recall_list_df_nolast[recall_list_df_nolast['user_id'].isin(click_trn_hist['user_id'].unique())]
    trn_user_item_label_df = get_rank_label_df(trn_user_items_df, click_trn_last, is_test=False)
    trn_user_item_label_df = neg_sample_recall_data(trn_user_item_label_df)
    if click_val is not None:
        val_user_items_df = recall_list_df_nolast[recall_list_df_nolast['user_id'].isin(click_val_hist['user_id'].unique())]
        val_user_item_label_df = get_rank_label_df(val_user_items_df, click_val_last, is_test=False)
    else:
        val_user_item_label_df = None
    tst_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_tst_hist['user_id'].unique())]
    tst_user_item_label_df = get_rank_label_df(tst_user_items_df, None, is_test=True)
    return trn_user_item_label_df, val_user_item_label_df, tst_user_item_label_df

#recall_list_df50 have 50 recall items
trn_user_item_label_df, val_user_item_label_df, tst_user_item_label_df = get_user_recall_item_label_df(click_trn_hist, 
                                          click_val_hist, 
                                          click_tst_hist,
                                          click_trn_last, 
                                          click_val_last, 
                                          recall_list_df50_nolast,
                                          recall_list_df50,
                                          click_val
                                          )


In [ ]:
def trian_item_word2vec(click_df, save_path, embed_size=64, save_name='item_w2v_emb.pkl', split_char=' '):
    click_df = click_df.sort_values('click_timestamp')
    click_df['click_article_id'] = click_df['click_article_id'].astype(str)
    docs = click_df.groupby(['user_id'])['click_article_id'].apply(lambda x: list(x)).reset_index()
    docs = docs['click_article_id'].values.tolist()
    logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)
    w2v = Word2Vec(docs, size=16, sg=1, window=5, seed=2020, workers=24, min_count=1, iter=1)
    item_w2v_emb_dict = {k: w2v[k] for k in click_df['click_article_id']}
    pickle.dump(item_w2v_emb_dict, open(save_path + 'item_w2v_emb.pkl', 'wb'))    
    return item_w2v_emb_dict

def get_embedding(save_path, all_click_df):
    if os.path.exists(save_path + 'item_content_emb.pkl'):
        item_content_emb_dict = pickle.load(open(save_path + 'item_content_emb.pkl', 'rb'))
    if os.path.exists(save_path + 'item_w2v_emb.pkl'):
        item_w2v_emb_dict = pickle.load(open(save_path + 'item_w2v_emb.pkl', 'rb'))
    else:
        item_w2v_emb_dict = trian_item_word2vec(all_click_df,save_path)          
    return item_content_emb_dict, item_w2v_emb_dict

def get_article_info_df(data_path,article_name='articles.csv'):
    article_info_df = pd.read_csv(data_path + article_name)
    article_info_df = reduce_mem(article_info_df)    
    return article_info_df
    
article_info_df = get_article_info_df(data_path,article_name=article_name)
all_click = click_trn.append(click_tst)
all_click = all_click.append(click_val)
item_content_emb_dict, item_w2v_emb_dict= get_embedding(save_path, all_click)

In [ ]:
len(click_trn['user_id'].unique())

In [ ]:
len(click_val['user_id'].unique())

In [ ]:
len(click_tst['user_id'].unique())

###Adding features to the candidates

In [ ]:

def make_tuple_func(group_df):
    row_data = []
    for name, row_df in group_df.iterrows():
        row_data.append((row_df['sim_item'], row_df['score'], row_df['label']))
    return row_data

if os.path.exists(save_path + 'trn_user_item_feats_df.csv'):
    pass
else:
    trn_user_item_label_tuples = trn_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
    trn_user_item_label_tuples_dict = dict(zip(trn_user_item_label_tuples['user_id'], trn_user_item_label_tuples[0]))
    trn_user_item_feats_df = create_feature(users_id = trn_user_item_label_tuples_dict.keys(), \
                         recall_list = trn_user_item_label_tuples_dict,\
                         click_hist_df = click_trn_hist, \
                         articles_info = article_info_df, \
                         articles_emb =item_content_emb_dict,\
                         user_emb=None, N=1)
    trn_user_item_feats_df.to_csv(save_path + 'trn_user_item_feats_df.csv', index=False)

if os.path.exists(save_path+'val_user_item_feats_df.csv'):
    pass
else:
    if val_user_item_label_df is not None:
        val_user_item_label_tuples = val_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
        val_user_item_label_tuples_dict = dict(zip(val_user_item_label_tuples['user_id'], val_user_item_label_tuples[0]))
        val_user_item_feats_df = create_feature(val_user_item_label_tuples_dict.keys(), val_user_item_label_tuples_dict,\
                          click_val_hist, article_info_df, item_content_emb_dict)
        val_user_item_feats_df.to_csv(save_path + 'val_user_item_feats_df.csv', index=False)
    else:
      val_user_item_label_df = None

if test_name=='testA_click_log.csv' and os.path.exists(save_path + 'tst_user_item_feats_df.csv'):
    pass
else:
    tst_user_item_label_tuples = tst_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
    tst_user_item_label_tuples_dict = dict(zip(tst_user_item_label_tuples['user_id'], tst_user_item_label_tuples[0]))
    tst_user_item_feats_df = create_feature(tst_user_item_label_tuples_dict.keys(), tst_user_item_label_tuples_dict,click_tst_hist, article_info_df, item_content_emb_dict)
    tst_user_item_feats_df.to_csv(save_path + 'tst_user_item_feats_df.csv', index=False) 

if test_name=='testA_click_log.csv' and os.path.exists(save_path + 'user_info.csv'):
    pass
else:
    articles =  pd.read_csv(data_path+article_name)
    articles = reduce_mem(articles)
    all_data = click_trn.append(click_tst)
    if click_val is not None:
        all_data = all_data.append(click_val)
    all_data = reduce_mem(all_data)
    all_data = all_data.merge(articles, left_on='click_article_id', right_on='article_id') 
    user_act_fea = active_level(all_data, ['user_id', 'click_article_id', 'click_timestamp'])
    article_hot_fea = hot_level(all_data, ['user_id', 'click_article_id', 'click_timestamp'])    
    device_cols = ['user_id', 'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type']
    user_device_info = device_fea(all_data, device_cols)
    user_time_hob_cols = ['user_id', 'click_timestamp', 'created_at_ts']
    user_time_hob_info = user_time_hob_fea(all_data, user_time_hob_cols)
    user_category_hob_cols = ['user_id', 'category_id']
    user_cat_hob_info = user_cat_hob_fea(all_data, user_category_hob_cols)
    user_wcou_info = all_data.groupby('user_id')['words_count'].agg('mean').reset_index()
    user_wcou_info.rename(columns={'words_count': 'words_hbo'}, inplace=True)
    user_info = pd.merge(user_act_fea, user_device_info, on='user_id')
    user_info = user_info.merge(user_time_hob_info, on='user_id')
    user_info = user_info.merge(user_cat_hob_info, on='user_id')
    user_info = user_info.merge(user_wcou_info, on='user_id')
    user_info.to_csv(save_path + user_info_name, index=False)   

user_info = pd.read_csv(save_path + user_info_name)
articles =  pd.read_csv(data_path+article_name)
articles = reduce_mem(articles)

if os.path.exists(save_path + 'trn_user_item_feats_df_finalversion.csv'):
    pass
else:
    if os.path.exists(save_path + 'trn_user_item_feats_df.csv'):
        trn_user_item_feats_df = pd.read_csv(save_path + 'trn_user_item_feats_df.csv')
    trn_user_item_feats_df = trn_user_item_feats_df.merge(user_info, on='user_id', how='left')
    trn_user_item_feats_df = trn_user_item_feats_df.merge(articles, left_on='click_article_id', right_on='article_id')
    trn_user_item_feats_df['is_cat_hab'] = trn_user_item_feats_df.apply(lambda x: 1 if x.category_id in set(x.cate_list) else 0, axis=1)
    del trn_user_item_feats_df['cate_list']
    del trn_user_item_feats_df['article_id']
    trn_user_item_feats_df.to_csv(save_path + 'trn_user_item_feats_df_finalversion.csv', index=False)

if os.path.exists(save_path + 'tst_user_item_feats_df_finalversion.csv'):
    pass
else:
    if os.path.exists(save_path + 'tst_user_item_feats_df.csv'):
        tst_user_item_feats_df = pd.read_csv(save_path + 'tst_user_item_feats_df.csv')
    tst_user_item_feats_df = tst_user_item_feats_df.merge(user_info, on='user_id',how='left') 
    tst_user_item_feats_df = tst_user_item_feats_df.merge(articles, left_on='click_article_id', right_on='article_id') 
    tst_user_item_feats_df['is_cat_hab'] = tst_user_item_feats_df.apply(lambda x: 1 if x.category_id in set(x.cate_list) else 0, axis=1)    
    del tst_user_item_feats_df['cate_list']
    del tst_user_item_feats_df['article_id']    
    tst_user_item_feats_df.to_csv(save_path + 'tst_user_item_feats_df_finalversion.csv', index=False)    

if os.path.exists(save_path+'val_user_item_feats_df_finalversion.csv') or not have_validation:
    val_user_item_feats_df=None
    pass
else:
    if os.path.exists(save_path + 'val_user_item_feats_df.csv'):
        val_user_item_feats_df = pd.read_csv(save_path + 'val_user_item_feats_df.csv')
    val_user_item_feats_df = val_user_item_feats_df.merge(user_info, on='user_id', how='left')
    val_user_item_feats_df = val_user_item_feats_df.merge(articles, left_on='click_article_id', right_on='article_id')
    val_user_item_feats_df['is_cat_hab'] = val_user_item_feats_df.apply(lambda x: 1 if x.category_id in set(x.cate_list) else 0, axis=1)
    del val_user_item_feats_df['cate_list']
    del val_user_item_feats_df['article_id']
    val_user_item_feats_df.to_csv(save_path + 'val_user_item_feats_df_finalversion.csv', index=False)




-- Mem. usage decreased to  5.56 Mb (50.0% reduction),time spend:0.00 min


In [ ]:
val_user_item_feats_df['user_id'].unique()[:100]

array([  33,   85,  107,  119,  129,  131,  133,  134,  144,  169,  237,
        243,  254,  256,  282,  283,  294,  301,  321,  335,  355,  385,
        396,  411,  438,  444,  452,  501,  512,  521,  557,  571,  609,
        646,  678,  679,  680,  750,  775,  788,  851,  870,  876,  886,
        916,  921,  940,  944,  962,  991, 1047, 1055, 1076, 1097, 1188,
       1194, 1195, 1217, 1223, 1225, 1228, 1230, 1239, 1244, 1341, 1369,
       1370, 1388, 1408, 1459, 1462, 1466, 1506, 1539, 1570, 1592, 1593,
       1594, 1603, 1639, 1641, 1650, 1669, 1693, 1717, 1733, 1835, 1848,
       1851, 1863, 1864, 1884, 1887, 1905, 1909, 1927, 1936, 1939, 1940,
       1951])

##Start model

###1. Load training and test data

In [ ]:
have_validation = False
trn_user_item_feats_df = pd.read_csv(save_path + 'trn_user_item_feats_df_finalversion.csv')
trn_user_item_feats_df['click_article_id'] = trn_user_item_feats_df['click_article_id'].astype(int)

val_user_item_feats_df = pd.read_csv(save_path + 'val_user_item_feats_df_finalversion.csv')
val_user_item_feats_df['click_article_id'] = val_user_item_feats_df['click_article_id'].astype(int)

tst_user_item_feats_df = pd.read_csv(save_path + 'tst_user_item_feats_df_finalversion.csv')
tst_user_item_feats_df['click_article_id'] = tst_user_item_feats_df['click_article_id'].astype(int)
del tst_user_item_feats_df['label']


In [ ]:
if have_validation:
  trn_user_item_feats_df_rank_model = trn_user_item_feats_df.copy()
  val_user_item_feats_df_rank_model = val_user_item_feats_df.copy() 
else:
  trn_user_item_feats_df_rank_model = pd.concat([trn_user_item_feats_df,val_user_item_feats_df]) 
  tst_user_item_feats_df_rank_model = tst_user_item_feats_df.copy()

###2. define features name

In [ ]:
lgb_cols = ['sim0', 'time_diff0', 'word_diff0','sim_max', 'sim_min', 'sim_sum', 
            'sim_mean', 'score','click_size', 'time_diff_mean', 'active_level',
            'click_environment','click_deviceGroup', 'click_os', 'click_country', 
            'click_region','click_referrer_type', 'user_time_hob1', 'user_time_hob2',
            'words_hbo', 'category_id', 'created_at_ts','words_count']
#'click_timestamp': 'user_time_hob1', 'created_at_ts': 'user_time_hob2'
#'words_hbo': words_count_mean

In [ ]:
def norm_sim(sim_df, weight=0.0):
    min_sim = sim_df.min()
    max_sim = sim_df.max()
    if max_sim == min_sim:
        sim_df = sim_df.apply(lambda sim: 1.0)
    else:
        sim_df = sim_df.apply(lambda sim: 1.0 * (sim - min_sim) / (max_sim - min_sim))
    sim_df = sim_df.apply(lambda sim: sim + weight)
    return sim_df

###3. lgb rank

In [ ]:
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['user_id'].unique()
    user_set = [user_ids[i::n] for i in range(n)]
    return user_set
k_fold = 5

trn_df = trn_user_item_feats_df_rank_model.copy()
user_set = get_kfold_users(trn_df, n=k_fold)
score_list = []
score_df = trn_df[['user_id', 'click_article_id','label']]

if have_validation:
  sub_preds_val = np.zeros(val_user_item_feats_df_rank_model.shape[0])
else:
  sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])
  
for n_fold, valid_user in enumerate(user_set):
    train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] 
    valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]
    train_idx.sort_values(by=['user_id'], inplace=True)
    g_train = train_idx.groupby(['user_id'], as_index=False).count()["label"].values
    valid_idx.sort_values(by=['user_id'], inplace=True)
    g_val = valid_idx.groupby(['user_id'], as_index=False).count()["label"].values
    lgb_ranker = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=1000, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16)  
    lgb_ranker.fit(train_idx[lgb_cols], train_idx['label'], group=g_train,
                   eval_set=[(valid_idx[lgb_cols], valid_idx['label'])], eval_group= [g_val], 
                   eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, )
    valid_idx['pred_score'] = lgb_ranker.predict(valid_idx[lgb_cols], num_iteration=lgb_ranker.best_iteration_)
    valid_idx['pred_score'] = valid_idx[['pred_score']].transform(lambda x: norm_sim(x))  
    valid_idx.sort_values(by=['user_id', 'pred_score'])
    valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])
    if not have_validation:
        sub_preds += lgb_ranker.predict(tst_user_item_feats_df_rank_model[lgb_cols], lgb_ranker.best_iteration_)
    if have_validation:
        sub_preds_val += lgb_ranker.predict(val_user_item_feats_df_rank_model[lgb_cols], lgb_ranker.best_iteration_)
score_df_ = pd.concat(score_list, axis=0)
score_df = score_df.merge(score_df_, how='left', on=['user_id', 'click_article_id'])
score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_lgb_ranker_feats.csv', index=False)
if not have_validation:
  tst_user_item_feats_df_rank_model['pred_score'] = sub_preds / k_fold
  tst_user_item_feats_df_rank_model['pred_score'] = tst_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))
  tst_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
  tst_user_item_feats_df_rank_model['pred_rank'] = tst_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
  tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_lgb_ranker_feats.csv', index=False)
else:
  val_user_item_feats_df_rank_model['pred_score'] = sub_preds_val / k_fold
  val_user_item_feats_df_rank_model['pred_score'] = val_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))
  val_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
  val_user_item_feats_df_rank_model['pred_rank'] = val_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
  val_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'val_lgb_ranker_feats.csv', index=False)



Streaming output truncated to the last 5000 lines.
[15]	valid_0's ndcg@1: 0.870475	valid_0's ndcg@2: 0.93442	valid_0's ndcg@3: 0.938445	valid_0's ndcg@4: 0.939844	valid_0's ndcg@5: 0.940628	valid_0's ndcg@1: 0.870475	valid_0's ndcg@2: 0.93442	valid_0's ndcg@3: 0.938445	valid_0's ndcg@4: 0.939844	valid_0's ndcg@5: 0.940628
[16]	valid_0's ndcg@1: 0.870425	valid_0's ndcg@2: 0.934685	valid_0's ndcg@3: 0.938598	valid_0's ndcg@4: 0.939933	valid_0's ndcg@5: 0.940755	valid_0's ndcg@1: 0.870425	valid_0's ndcg@2: 0.934685	valid_0's ndcg@3: 0.938598	valid_0's ndcg@4: 0.939933	valid_0's ndcg@5: 0.940755
[17]	valid_0's ndcg@1: 0.870275	valid_0's ndcg@2: 0.934598	valid_0's ndcg@3: 0.938461	valid_0's ndcg@4: 0.939871	valid_0's ndcg@5: 0.940606	valid_0's ndcg@1: 0.870275	valid_0's ndcg@2: 0.934598	valid_0's ndcg@3: 0.938461	valid_0's ndcg@4: 0.939871	valid_0's ndcg@5: 0.940606
[18]	valid_0's ndcg@1: 0.869775	valid_0's ndcg@2: 0.934382	valid_0's ndcg@3: 0.938295	valid_0's ndcg@4: 0.939673	valid_0's ndc

In [ ]:
trn_user_item_feats_df_rank_model['label'].value_counts()

0.0    712541
1.0    148020
Name: label, dtype: int64

### 4. lgb classifier

In [ ]:
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['user_id'].unique()
    user_set = [user_ids[i::n] for i in range(n)]
    return user_set
k_fold = 5
trn_df = trn_user_item_feats_df_rank_model.copy()
user_set = get_kfold_users(trn_df, n=k_fold)
score_list = []
score_df = trn_df[['user_id', 'click_article_id', 'label']]

if have_validation:
  sub_preds_val = np.zeros(val_user_item_feats_df_rank_model.shape[0])
else:
  sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])
for n_fold, valid_user in enumerate(user_set):
    train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] 
    valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]
    lgb_Classfication = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=1000, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16, verbose=10)  
    lgb_Classfication.fit(train_idx[lgb_cols], train_idx['label'],eval_set=[(valid_idx[lgb_cols], valid_idx['label'])], 
                          eval_metric=['auc', ],early_stopping_rounds=50, )
    valid_idx['pred_score'] = lgb_Classfication.predict_proba(valid_idx[lgb_cols], num_iteration=lgb_Classfication.best_iteration_)[:,1]
    valid_idx.sort_values(by=['user_id', 'pred_score'])
    valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])
    if not have_validation:
        sub_preds += lgb_Classfication.predict_proba(tst_user_item_feats_df_rank_model[lgb_cols],num_iteration=lgb_Classfication.best_iteration_)[:,1]
    if have_validation:
        sub_preds_val += lgb_Classfication.predict_proba(val_user_item_feats_df_rank_model[lgb_cols],num_iteration=lgb_Classfication.best_iteration_)[:,1]
score_df_ = pd.concat(score_list, axis=0)
score_df = score_df.merge(score_df_, how='left', on=['user_id', 'click_article_id'])
score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_lgb_cls_feats.csv', index=False)
if not have_validation:
  tst_user_item_feats_df_rank_model['pred_score'] = sub_preds / k_fold
  tst_user_item_feats_df_rank_model['pred_score'] = tst_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))
  tst_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
  tst_user_item_feats_df_rank_model['pred_rank'] = tst_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
  tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_lgb_cls_feats.csv', index=False)
else:
  val_user_item_feats_df_rank_model['pred_score'] = sub_preds_val / k_fold
  val_user_item_feats_df_rank_model['pred_score'] = val_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))
  val_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
  val_user_item_feats_df_rank_model['pred_rank'] = val_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
  val_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'val_lgb_cls_feats.csv', index=False)




Streaming output truncated to the last 5000 lines.
[15]	valid_0's binary_logloss: 0.42948	valid_0's auc: 0.863529
[16]	valid_0's binary_logloss: 0.428248	valid_0's auc: 0.86378
[17]	valid_0's binary_logloss: 0.42642	valid_0's auc: 0.863528
[18]	valid_0's binary_logloss: 0.425339	valid_0's auc: 0.863338
[19]	valid_0's binary_logloss: 0.423757	valid_0's auc: 0.863877
[20]	valid_0's binary_logloss: 0.421984	valid_0's auc: 0.863908
[21]	valid_0's binary_logloss: 0.42164	valid_0's auc: 0.86413
[22]	valid_0's binary_logloss: 0.42056	valid_0's auc: 0.864101
[23]	valid_0's binary_logloss: 0.419536	valid_0's auc: 0.863782
[24]	valid_0's binary_logloss: 0.417874	valid_0's auc: 0.863727
[25]	valid_0's binary_logloss: 0.416262	valid_0's auc: 0.863564
[26]	valid_0's binary_logloss: 0.415262	valid_0's auc: 0.863514
[27]	valid_0's binary_logloss: 0.413687	valid_0's auc: 0.863456
[28]	valid_0's binary_logloss: 0.412655	valid_0's auc: 0.863458
[29]	valid_0's binary_logloss: 0.411153	valid_0's auc: 0.86

###DIN


In [ ]:
# 导入deepctr
from deepctr.models import DIN
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [ ]:
have_validation = True
if have_validation:
    all_data = pd.read_csv('./tcdata/train_click_log.csv')
else:
    trn_data = pd.read_csv('./tcdata/train_click_log.csv')
    tst_data = pd.read_csv('./tcdata/testA_click_log.csv')
    all_data = trn_data.append(tst_data)

In [ ]:
hist_click =all_data[['user_id', 'click_article_id']].groupby('user_id').agg({list}).reset_index()
his_behavior_df = pd.DataFrame()
his_behavior_df['user_id'] = hist_click['user_id']
his_behavior_df['hist_click_article_id'] = hist_click['click_article_id']

In [ ]:
if have_validation:
  trn_user_item_feats_df_din_model = trn_user_item_feats_df.copy()
  val_user_item_feats_df_din_model = val_user_item_feats_df.copy()
else:
  trn_user_item_feats_df_din_model = pd.concat([trn_user_item_feats_df,val_user_item_feats_df])
  tst_user_item_feats_df_din_model = tst_user_item_feats_df.copy()

In [ ]:
trn_user_item_feats_df_din_model = trn_user_item_feats_df_din_model.merge(his_behavior_df, on='user_id')

if have_validation:
    val_user_item_feats_df_din_model = val_user_item_feats_df_din_model.merge(his_behavior_df, on='user_id')
else:
    val_user_item_feats_df_din_model = None
    tst_user_item_feats_df_din_model = tst_user_item_feats_df_din_model.merge(his_behavior_df, on='user_id')

In [ ]:
if have_validation:
  click_val = all_data[all_data['user_id'].isin(val_user_item_feats_df['user_id'].unique())]
  click_val = click_val.sort_values(['user_id', 'click_timestamp'])
  click_val_last = click_val.groupby('user_id').tail(1)

In [ ]:
# 数据准备函数
def get_din_feats_columns(df, dense_fea, sparse_fea, behavior_fea, hist_behavior_fea, feature_max_idx,emb_dim=64, max_len=100):
    """
    数据准备函数:
    df: 数据集
    dense_fea: 数值型特征列
    sparse_fea: 离散型特征列
    behavior_fea: 用户的候选行为特征列
    hist_behavior_fea: 用户的历史行为特征列
    embedding_dim: embedding的维度， 这里为了简单， 统一把离散型特征列采用一样的隐向量维度
    max_len: 用户序列的最大长度
    """

    sparse_feature_columns = [SparseFeat(feat, feature_max_idx[feat], embedding_dim=emb_dim) for feat in sparse_fea]
    
    dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_fea]
    
    var_feature_columns = [VarLenSparseFeat(SparseFeat(feat, feature_max_idx['click_article_id'],\
                embedding_dim=emb_dim, embedding_name='click_article_id'), maxlen=max_len) for feat in hist_behavior_fea]
    
    dnn_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns
    
    # 建立x, x是一个字典的形式
    x = {}
    for name in get_feature_names(dnn_feature_columns):
        if name in hist_behavior_fea:
            # 这是历史行为序列
            his_list = [l for l in df[name]]
            x[name] = pad_sequences(his_list, maxlen=max_len, padding='post')      # 二维数组
        else:
            x[name] = df[name].values
    
    return x, dnn_feature_columns

In [ ]:
# 把特征分开
sparse_fea = ['user_id', 'click_article_id', 'category_id', 'click_environment', 'click_deviceGroup', 
              'click_os', 'click_country', 'click_region', 'click_referrer_type', 'is_cat_hab']

behavior_fea = ['click_article_id']

hist_behavior_fea = ['hist_click_article_id']

dense_fea = ['sim0', 'time_diff0', 'word_diff0', 'sim_max', 'sim_min', 'sim_sum', 'sim_mean', 'score',
             'rank','click_size','time_diff_mean','active_level','user_time_hob1','user_time_hob2',
             'words_hbo','words_count']

In [ ]:
# dense特征进行归一化, 神经网络训练都需要将数值进行归一化处理
mm = MinMaxScaler()

# 下面是做一些特殊处理，当在其他的地方出现无效值的时候，不处理无法进行归一化，刚开始可以先把他注释掉，在运行了下面的代码
# 之后如果发现报错，应该先去想办法处理如何不出现inf之类的值

# trn_user_item_feats_df_din_model.replace([np.inf, -np.inf], 0, inplace=True)
# if have_validation:
#   val_user_item_feats_df_din_model.replace([np.inf, -np.inf], 0, inplace=True)
# else:
#   tst_user_item_feats_df_din_model.replace([np.inf, -np.inf], 0, inplace=True)

for feat in dense_fea:
    trn_user_item_feats_df_din_model[feat] = mm.fit_transform(trn_user_item_feats_df_din_model[[feat]])
    
    if have_validation:
      val_user_item_feats_df_din_model[feat] = mm.fit_transform(val_user_item_feats_df_din_model[[feat]])
    else:
      tst_user_item_feats_df_din_model[feat] = mm.fit_transform(tst_user_item_feats_df_din_model[[feat]])


In [ ]:
if have_validation:
  all_data_df = pd.concat([trn_user_item_feats_df_din_model,val_user_item_feats_df_din_model])
else:
  all_data_df = pd.concat([trn_user_item_feats_df_din_model,tst_user_item_feats_df_din_model])

In [ ]:
feature_max_idx = {}
for feature in sparse_fea:
  feature_max_idx[feature] = all_data_df[feature].max() + 1


In [ ]:
# 准备训练数据
x_trn, dnn_feature_columns = get_din_feats_columns(trn_user_item_feats_df_din_model, dense_fea,\
              sparse_fea, behavior_fea, hist_behavior_fea, feature_max_idx, emb_dim = 64, max_len=50,)
y_trn = trn_user_item_feats_df_din_model['label'].values

dense_fea = [x for x in dense_fea if x != 'label']
if have_validation:
    # 准备验证数据
  x_val, dnn_feature_columns = get_din_feats_columns(val_user_item_feats_df_din_model, dense_fea, \
                   sparse_fea, behavior_fea, hist_behavior_fea, feature_max_idx, max_len=50)
  y_val = val_user_item_feats_df_din_model['label'].values
else:
  x_tst, dnn_feature_columns = get_din_feats_columns(tst_user_item_feats_df_din_model, dense_fea,\
                         sparse_fea, behavior_fea, hist_behavior_fea, feature_max_idx, max_len=50)

In [ ]:
# 建立模型
model = DIN(dnn_feature_columns, behavior_fea)

# 查看模型结构
model.summary()

# 模型编译
model.compile('adam', 'binary_crossentropy',metrics=['binary_crossentropy', tf.keras.metrics.AUC()])

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'attention_sequence_pooling_layer/local_activation_unit/kernel:0' shape=(40, 1) dtype=float32>
  <tf.Variable 'attention_sequence_pooling_layer/local_activation_unit/bias:0' shape=(1,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'attention_sequence_pooling_layer/local_activation_unit/kernel:0' shape=(40, 1) dtype=float32>
  <tf.Variable 'attention_sequence_pooling_layer/local_activation_unit/bias:0' shape=(1,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
click_article_id (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
category_id (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
click_environment (InputLayer)  [(None, 1)]          0                                            
______________________________________________________________________________________________

In [ ]:
# 模型训练
# 也可以使用下面的语句用自己采样出来的验证集

# history = model.fit(x_trn, y_trn, verbose=1, epochs=10, validation_data=(x_val, y_val) , batch_size=256)
# history = model.fit(x_trn, y_trn, verbose=1, epochs=10, validation_split=0.3, batch_size=256)
# history = model.fit(x_trn, y_trn, verbose=1, epochs=2, batch_size=256)
# 预测：


In [ ]:
# 五折交叉验证，这里的五折交叉是以用户为目标进行五折划分
#  这一部分与前面的单独训练和验证是分开的
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['user_id'].unique()
    user_set = [user_ids[i::n] for i in range(n)]
    return user_set

k_fold = 5
trn_df = trn_user_item_feats_df_din_model
user_set = get_kfold_users(trn_df, n=k_fold)

score_list = []
score_df = trn_df[['user_id', 'click_article_id', 'label']]
if have_validation:
  sub_preds_val = np.zeros(val_user_item_feats_df_rank_model.shape[0])
else:
  sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])

dense_fea = [x for x in dense_fea if x != 'label']
if have_validation:
  x_val, dnn_feature_columns = get_din_feats_columns(val_user_item_feats_df_din_model, dense_fea,\
                       sparse_fea, behavior_fea, hist_behavior_fea, feature_max_idx, max_len=50)
else:
  x_tst, dnn_feature_columns = get_din_feats_columns(tst_user_item_feats_df_din_model, dense_fea,\
                       sparse_fea, behavior_fea, hist_behavior_fea, feature_max_idx, max_len=50)

# 五折交叉验证，并将中间结果保存用于staking
for n_fold, valid_user in enumerate(user_set):
    train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] # add slide user
    valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]
    
    # 准备训练数据
    x_trn_split, dnn_feature_columns = get_din_feats_columns(train_idx, dense_fea,\
                    sparse_fea, behavior_fea, hist_behavior_fea,feature_max_idx, max_len=50)
    y_trn_split = train_idx['label'].values

    # 准备验证数据
    x_val_split, dnn_feature_columns = get_din_feats_columns(valid_idx, dense_fea,\
                     sparse_fea, behavior_fea, hist_behavior_fea,feature_max_idx, max_len=50)
    y_val_split = valid_idx['label'].values
    
    history = model.fit(x_trn_split, y_trn_split, verbose=1, epochs=10, validation_data=(x_val_split, y_val_split) , batch_size=256)
    
    # 预测验证集结果
    valid_idx['pred_score'] = model.predict(x_val_split, verbose=1, batch_size=256)   
    
    valid_idx.sort_values(by=['user_id', 'pred_score'])
    valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 将验证集的预测结果放到一个列表中，后面进行拼接
    score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])
    
    # 如果是线上测试，需要计算每次交叉验证的结果相加，最后求平均
    if not have_validation:
      sub_preds += model.predict(x_tst, verbose=1, batch_size=256)[:, 0]
    else:
      sub_preds_val += model.predict(x_val, verbose=1, batch_size=256)[:, 0]
    
score_df_ = pd.concat(score_list, axis=0)
score_df = score_df.merge(score_df_, how='left', on=['user_id', 'click_article_id'])
# 保存训练集交叉验证产生的新特征
score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_din_cls_feats.csv', index=False)
    
# 测试集的预测结果，多次交叉验证求平均,将预测的score和对应的rank特征保存，可以用于后面的staking，这里还可以构造其他更多的特征
if not have_validation:
  tst_user_item_feats_df_din_model['pred_score'] = sub_preds / k_fold
  tst_user_item_feats_df_din_model['pred_score'] = tst_user_item_feats_df_din_model['pred_score'].transform(lambda x: norm_sim(x))
  tst_user_item_feats_df_din_model.sort_values(by=['user_id', 'pred_score'])
  tst_user_item_feats_df_din_model['pred_rank'] = tst_user_item_feats_df_din_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

  # 保存测试集交叉验证的新特征
  tst_user_item_feats_df_din_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_din_cls_feats.csv', index=False)
else:
  val_user_item_feats_df_din_model['pred_score'] = sub_preds_val / k_fold
  val_user_item_feats_df_din_model['pred_score'] = val_user_item_feats_df_din_model['pred_score'].transform(lambda x: norm_sim(x))
  val_user_item_feats_df_din_model.sort_values(by=['user_id', 'pred_score'])
  val_user_item_feats_df_din_model['pred_rank'] = val_user_item_feats_df_din_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
  val_user_item_feats_df_din_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'val_din_cls_feats.csv', index=False)



Epoch 1/10
1128/1128 [==============================] - 151s 130ms/step - loss: 0.3973 - binary_crossentropy: 0.3964 - auc: 0.8670 - val_loss: 0.0504 - val_binary_crossentropy: 0.0468 - val_auc: 0.9985
Epoch 2/10
1128/1128 [==============================] - 146s 130ms/step - loss: 0.0283 - binary_crossentropy: 0.0247 - auc: 0.9993 - val_loss: 0.0380 - val_binary_crossentropy: 0.0342 - val_auc: 0.9990
Epoch 3/10
1128/1128 [==============================] - 146s 130ms/step - loss: 0.0068 - binary_crossentropy: 0.0032 - auc: 0.9999 - val_loss: 0.0364 - val_binary_crossentropy: 0.0332 - val_auc: 0.9984
Epoch 4/10
1128/1128 [==============================] - 147s 131ms/step - loss: 0.0043 - binary_crossentropy: 0.0013 - auc: 1.0000 - val_loss: 0.0465 - val_binary_crossentropy: 0.0439 - val_auc: 0.9972
Epoch 5/10
1128/1128 [==============================] - 145s 128ms/step - loss: 0.0044 - binary_crossentropy: 0.0020 - auc: 1.0000 - val_loss: 0.0527 - val_binary_crossentropy: 0.0503 - val_au

###Weighted Result

In [ ]:
trn_lgb_ranker_feats['user_id'].unique()[:30]

array([     0,    285,   1003,   2083, 130003,      1,   1710,   6757,
        23411,  61695,  63522,  72372, 124646, 176477, 178863,   1454,
         1557,   1711,   3896,      2,    138,    182,    467,    942,
         1200,   1357,   1410,   1619,   1659,   1678])

In [ ]:
tst_lgb_ranker_feats['user_id'].unique()[:30]

array([200000, 200004, 200005, 200015, 200020, 200022, 200023, 200024,
       200035, 200036, 200037, 200044, 200050, 200053, 200055, 200061,
       200067, 200068, 200073, 200076, 200081, 200091, 200096, 200097,
       200101, 200103, 200105, 200112, 200117, 200121])

In [ ]:
have_validation=False

In [ ]:
trn_lgb_ranker_feats

,user_id,click_article_id,pred_score,pred_rank,label
0,0,265004,0.462911,2.0,0.0
1,0,111338,0.556843,1.0,0.0
2,285,111338,0.490005,2.0,0.0
3,1003,111338,0.458856,2.0,0.0
4,2083,111338,0.370019,2.0,0.0
...,...,...,...,...,...
860556,199740,211484,0.312032,28.0,0.0
860557,199740,272762,0.503382,8.0,0.0
860558,199740,140898,0.186603,43.0,0.0
860559,199740,128881,0.290743,33.0,0.0


In [ ]:
#加权融合
import pandas as pd
trn_lgb_ranker_feats = pd.read_csv(save_path + 'trn_lgb_ranker_feats.csv')
trn_lgb_cls_feats = pd.read_csv(save_path + 'trn_lgb_cls_feats.csv')
# trn_din_cls_feats = pd.read_csv(save_path + 'trn_din_cls_feats.csv')
if not have_validation:
  tst_lgb_ranker_feats = pd.read_csv(save_path + 'tst_lgb_ranker_feats.csv')
  tst_lgb_cls_feats = pd.read_csv(save_path + 'tst_lgb_cls_feats.csv')
  # tst_din_cls_feats = pd.read_csv(save_path + 'tst_din_cls_feats.csv')
else:
  val_lgb_ranker_feats = pd.read_csv(save_path + 'val_lgb_ranker_feats.csv')
  val_lgb_cls_feats = pd.read_csv(save_path + 'val_lgb_cls_feats.csv')
  # val_din_cls_feats = pd.read_csv(save_path + 'val_din_cls_feats.csv')

finall_trn_ranker_feats = trn_lgb_ranker_feats[['user_id', 'click_article_id', 'label']]
if not have_validation:
  finall_tst_ranker_feats = tst_lgb_ranker_feats[['user_id', 'click_article_id']]
if have_validation:
  finall_val_ranker_feats = val_lgb_ranker_feats[['user_id','click_article_id']]
  
for idx, trn_model in enumerate([trn_lgb_ranker_feats, trn_lgb_cls_feats]):
    for feat in [ 'pred_score', 'pred_rank']:
        col_name = feat + '_' + str(idx)
        finall_trn_ranker_feats[col_name] = trn_model[feat]
if not have_validation:
  for idx, tst_model in enumerate([tst_lgb_ranker_feats, tst_lgb_cls_feats]):
    for feat in [ 'pred_score', 'pred_rank']:
        col_name = feat + '_' + str(idx)
        finall_tst_ranker_feats[col_name] = tst_model[feat]
else:
  for idx, val_model in enumerate([val_lgb_ranker_feats, val_lgb_cls_feats]):
      for feat in [ 'pred_score', 'pred_rank']:
          col_name = feat + '_' + str(idx)
          finall_val_ranker_feats[col_name] = val_model[feat]

feat_cols = ['pred_score_0', 'pred_rank_0', 'pred_score_1', 'pred_rank_1']
trn_x = finall_trn_ranker_feats[feat_cols]
trn_y = finall_trn_ranker_feats['label']
if not have_validation:
  tst_x = finall_tst_ranker_feats[feat_cols]
else:
  val_x = finall_val_ranker_feats[feat_cols]

trn_x.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
logistic_regression = True
gbdt = False
if logistic_regression:
  lr = LogisticRegression()
  lr.fit(trn_x, trn_y)
  if not have_validation:
    finall_tst_ranker_feats['pred_score'] = lr.predict_proba(tst_x)[:, 1]
  else:
    finall_val_ranker_feats['pred_score'] = lr.predict_proba(val_x)[:, 1]

from sklearn.ensemble import GradientBoostingRegressor
if gbdt:
  regressor = GradientBoostingRegressor(
      max_depth=3,
      n_estimators=300,
      learning_rate=0.001
  )
  regressor.fit(trn_x, trn_y)
  if not have_validation:
    finall_tst_ranker_feats['pred_score'] = regressor.predict(tst_x)
  else:
    finall_val_ranker_feats['pred_score'] = regressor.predict(val_x)

if not have_validation:
  rank_results = finall_tst_ranker_feats[['user_id', 'click_article_id', 'pred_score']]
else:
  rank_results_val = finall_val_ranker_feats[['user_id', 'click_article_id', 'pred_score']]

In [ ]:
val_lgb_ranker_feats = pd.read_csv(save_path + 'val_lgb_ranker_feats.csv')
val_lgb_cls_feats = pd.read_csv(save_path + 'val_lgb_cls_feats.csv')
val_din_cls_feats = pd.read_csv(save_path + 'val_din_cls_feats.csv')

###Metric if have validation

In [ ]:
def get_submit(result, all_data):
  recall_df = result.sort_values(by=['user_id', 'pred_score'],ascending=True)
  recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
  tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
  assert tmp.min()>=5
  submit = recall_df.sort_values(by=['user_id','rank'],ascending=True)
  test = submit.groupby('user_id')['click_article_id'].apply(lambda x:[i for i in x])
  test2 = test.reset_index()
  submit_dict = {}
  submit_dict = dict(zip(test2['user_id'], test2['click_article_id']))

  click_val = all_data[all_data['user_id'].isin(result['user_id'].unique())]
  click_val = click_val.sort_values(['user_id', 'click_timestamp'])
  click_val_last = click_val.groupby('user_id').tail(1)
  return submit_dict, click_val_last


In [ ]:
def metrics_recall2(user_recall_items_dict, trn_last_click_df, topk=50):
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['click_article_id']))
    user_num = len(user_recall_items_dict) 
    for k in range(5, topk+1, 5):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            tmp_recall_items = user_recall_items_dict[user][:k]
            if last_click_item_dict[user] in set(tmp_recall_items):
                hit_num += 1        
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)

In [ ]:
# submit_dict, click_val_last = get_submit(val_lgb_ranker_feats, all_data)
# metrics_recall2(submit_dict, click_val_last)
#new recall hot+simi+hot
#  topk:  5  :  hit_num:  4974 hit_rate:  0.4974 user_num :  10000
#  topk:  10  :  hit_num:  6214 hit_rate:  0.6214 user_num :  10000
#  topk:  15  :  hit_num:  6776 hit_rate:  0.6776 user_num :  10000
#  topk:  20  :  hit_num:  7057 hit_rate:  0.7057 user_num :  10000
#  topk:  25  :  hit_num:  7194 hit_rate:  0.7194 user_num :  10000
#  topk:  30  :  hit_num:  7268 hit_rate:  0.7268 user_num :  10000
#  topk:  35  :  hit_num:  7306 hit_rate:  0.7306 user_num :  10000
#  topk:  40  :  hit_num:  7332 hit_rate:  0.7332 user_num :  10000
#  topk:  45  :  hit_num:  7347 hit_rate:  0.7347 user_num :  10000
#  topk:  50  :  hit_num:  7347 hit_rate:  0.7347 user_num :  10000

In [ ]:
# submit_dict, click_val_last = get_submit(val_lgb_cls_feats, all_data)
# metrics_recall2(submit_dict, click_val_last)
#new recall hot+simi+hot
#  topk:  5  :  hit_num:  4979 hit_rate:  0.4979 user_num :  10000
#  topk:  10  :  hit_num:  6210 hit_rate:  0.621 user_num :  10000
#  topk:  15  :  hit_num:  6771 hit_rate:  0.6771 user_num :  10000
#  topk:  20  :  hit_num:  7056 hit_rate:  0.7056 user_num :  10000
#  topk:  25  :  hit_num:  7190 hit_rate:  0.719 user_num :  10000
#  topk:  30  :  hit_num:  7264 hit_rate:  0.7264 user_num :  10000
#  topk:  35  :  hit_num:  7304 hit_rate:  0.7304 user_num :  10000
#  topk:  40  :  hit_num:  7336 hit_rate:  0.7336 user_num :  10000
#  topk:  45  :  hit_num:  7347 hit_rate:  0.7347 user_num :  10000
#  topk:  50  :  hit_num:  7347 hit_rate:  0.7347 user_num :  10000

In [ ]:
# submit_dict, click_val_last = get_submit(val_din_cls_feats, all_data)
# metrics_recall2(submit_dict, click_val_last)
#new recall hot+simi+hot
# topk:  5  :  hit_num:  1747 hit_rate:  0.1747 user_num :  10000
#  topk:  10  :  hit_num:  1867 hit_rate:  0.1867 user_num :  10000
#  topk:  15  :  hit_num:  1909 hit_rate:  0.1909 user_num :  10000
#  topk:  20  :  hit_num:  2414 hit_rate:  0.2414 user_num :  10000
#  topk:  25  :  hit_num:  3866 hit_rate:  0.3866 user_num :  10000
#  topk:  30  :  hit_num:  5019 hit_rate:  0.5019 user_num :  10000
#  topk:  35  :  hit_num:  5857 hit_rate:  0.5857 user_num :  10000
#  topk:  40  :  hit_num:  6522 hit_rate:  0.6522 user_num :  10000
#  topk:  45  :  hit_num:  6974 hit_rate:  0.6974 user_num :  10000
#  topk:  50  :  hit_num:  7347 hit_rate:  0.7347 user_num :  10000

In [ ]:
# submit_dict, click_val_last = get_submit(rank_results_val, all_data)
# metrics_recall2(submit_dict, click_val_last)
#new recall hot+simi+hot
#  topk:  5  :  hit_num:  4992 hit_rate:  0.4992 user_num :  10000
#  topk:  10  :  hit_num:  6210 hit_rate:  0.621 user_num :  10000
#  topk:  15  :  hit_num:  6775 hit_rate:  0.6775 user_num :  10000
#  topk:  20  :  hit_num:  7067 hit_rate:  0.7067 user_num :  10000
#  topk:  25  :  hit_num:  7197 hit_rate:  0.7197 user_num :  10000
#  topk:  30  :  hit_num:  7268 hit_rate:  0.7268 user_num :  10000
#  topk:  35  :  hit_num:  7308 hit_rate:  0.7308 user_num :  10000
#  topk:  40  :  hit_num:  7332 hit_rate:  0.7332 user_num :  10000
#  topk:  45  :  hit_num:  7347 hit_rate:  0.7347 user_num :  10000
#  topk:  50  :  hit_num:  7347 hit_rate:  0.7347 user_num :  10000

In [ ]:
# metrics_recall2(submit_dict, click_val_last) din
#  topk:  5  :  hit_num:  3859 hit_rate:  0.3859 user_num :  10000
#  topk:  10  :  hit_num:  4879 hit_rate:  0.4879 user_num :  10000
#  topk:  15  :  hit_num:  5615 hit_rate:  0.5615 user_num :  10000
#  topk:  20  :  hit_num:  6106 hit_rate:  0.6106 user_num :  10000
#  topk:  25  :  hit_num:  6540 hit_rate:  0.654 user_num :  10000
#  topk:  30  :  hit_num:  6902 hit_rate:  0.6902 user_num :  10000
#  topk:  35  :  hit_num:  7201 hit_rate:  0.7201 user_num :  10000
#  topk:  40  :  hit_num:  7429 hit_rate:  0.7429 user_num :  10000
#  topk:  45  :  hit_num:  7554 hit_rate:  0.7554 user_num :  10000
#  topk:  50  :  hit_num:  7636 hit_rate:  0.7636 user_num :  10000

In [ ]:
# metrics_recall2(submit_dict, click_val_last)  lgb rank
#  topk:  5  :  hit_num:  4438 hit_rate:  0.4438 user_num :  10000
#  topk:  10  :  hit_num:  5830 hit_rate:  0.583 user_num :  10000
#  topk:  15  :  hit_num:  6513 hit_rate:  0.6513 user_num :  10000
#  topk:  20  :  hit_num:  6944 hit_rate:  0.6944 user_num :  10000
#  topk:  25  :  hit_num:  7206 hit_rate:  0.7206 user_num :  10000
#  topk:  30  :  hit_num:  7375 hit_rate:  0.7375 user_num :  10000
#  topk:  35  :  hit_num:  7492 hit_rate:  0.7492 user_num :  10000
#  topk:  40  :  hit_num:  7595 hit_rate:  0.7595 user_num :  10000
#  topk:  45  :  hit_num:  7641 hit_rate:  0.7641 user_num :  10000
#  topk:  50  :  hit_num:  7669 hit_rate:  0.7669 user_num :  10000

###Submit

In [ ]:

def submit(pred_path, recall_df, topk=5):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    save_name = pred_path + 'result.csv'
    submit.to_csv(save_name, index=False, header=True)

submit(pred_path, rank_results, topk=5)

In [ ]:
# recall_df = rank_results_val.sort_values(by=['user_id', 'pred_score'],ascending=True)
# recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
# tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
# assert tmp.min()>=5
# submit = recall_df.sort_values(by=['user_id','rank'],ascending=True)
# test = submit.groupby('user_id')['click_article_id'].apply(lambda x:[i for i in x])
# test2 = test.reset_index()
# submit_dict = {}
# submit_dict = dict(zip(test2['user_id'], test2['click_article_id']))

In [ ]:
#LEARNING RATE = 0.01
# metrics_recall2(submit_dict, click_val_last)

In [ ]:
#LEARNING RATE = 0.001
# metrics_recall2(submit_dict, click_val_last)

In [ ]:
#LEARNING RATE = 0.001 N_ESTIMATEOR = 1000
# metrics_recall2(submit_dict, click_val_last)

In [ ]:
#LEARNING RATE = 0.01 N_ESTIMATEOR = 1000
# metrics_recall2(submit_dict, click_val_last)
#  topk:  5  :  hit_num:  5079 hit_rate:  0.5079 user_num :  10000
#  topk:  10  :  hit_num:  6288 hit_rate:  0.6288 user_num :  10000
#  topk:  15  :  hit_num:  6910 hit_rate:  0.691 user_num :  10000
#  topk:  20  :  hit_num:  7223 hit_rate:  0.7223 user_num :  10000
#  topk:  25  :  hit_num:  7405 hit_rate:  0.7405 user_num :  10000
#  topk:  30  :  hit_num:  7529 hit_rate:  0.7529 user_num :  10000
#  topk:  35  :  hit_num:  7592 hit_rate:  0.7592 user_num :  10000
#  topk:  40  :  hit_num:  7639 hit_rate:  0.7639 user_num :  10000
#  topk:  45  :  hit_num:  7664 hit_rate:  0.7664 user_num :  10000
#  topk:  50  :  hit_num:  7669 hit_rate:  0.7669 user_num :  10000

#Results:
for valdata: 原输出结果  
 topk:  5  :  hit_num:  4408 hit_rate:  0.4408 user_num :  10000  
 topk:  10  :  hit_num:  5747 hit_rate:  0.5747 user_num :  10000  
 topk:  15  :  hit_num:  6486 hit_rate:  0.6486 user_num :  10000  
 topk:  20  :  hit_num:  6891 hit_rate:  0.6891 user_num :  10000  
 topk:  25  :  hit_num:  7159 hit_rate:  0.7159 user_num :  10000  
 topk:  30  :  hit_num:  7344 hit_rate:  0.7344 user_num :  10000  
 topk:  35  :  hit_num:  7482 hit_rate:  0.7482 user_num :  10000  
 topk:  40  :  hit_num:  7564 hit_rate:  0.7564 user_num :  10000  
 topk:  45  :  hit_num:  7625 hit_rate:  0.7625 user_num :  10000  
 topk:  50  :  hit_num:  7657 hit_rate:  0.7657 user_num :  10000  